In [5]:
from ipyfilechooser import FileChooser

# Create and display a FileChooser widget
fc = FileChooser('/home/jovyan')

import pandas as pd

input = pd.read_table(fc.selected_filename,delimiter=',')
print('Check if this is the right file')
input

FileChooser(path='/home/jovyan', filename='', show_hidden='False')

In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
from sklearn.ensemble import RandomForestRegressor
import ipywidgets as widgets
from ipywidgets import interact
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model
import statsmodels.api as sm

combi = pd.read_table('Historical_combi2.csv',delimiter =',')

import datetime
datetimes = [datetime.datetime.strptime(d, '%d/%m/%Y') for d in combi["Date"]]


df = pd.DataFrame(datetimes, columns=['date'])
df.loc[:, 'date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
combi = combi.assign(df=df['date'].values)

pd.plotting.register_matplotlib_converters()

input = pd.read_table('inputdata.csv',delimiter=',')

DB = input["Dated Brent"]
Po = input["Propane"]
Gasoline = input["Gasoline"]
Jet = input["Jet"]
Go = input["Gasoil"]
FO1 = input["FO 1%"]
ULSD = input["ULSD"]
Na = input["Naphtha"]
Bu = input["Butane"]
FO35 = input["FO 3.5%"]
VGO05 = input["VGO 0.5%"]
FO05 = input["FO 0.5%"]
VGO2 = input["VGO 2%"]


def esc(code):
    return f'\033[{code}m'

def plotdiag(combi,model):
    
    model = model.replace("_", " ")
    plt.rcParams['figure.dpi'] = 100
    rma60 = combi[model].rolling(window=60).mean()
    ema60 = combi[model].ewm(span=60, adjust=False).mean()
    plt.plot(combi["df"],combi[model])
    plt.plot(combi["df"],rma60)
    plt.plot(combi["df"],ema60)
    plt.legend(("Monthly","rma","ema"))
    plt.show()
    output = [combi['df'],rma60,ema60]
    output_all = pd.concat(output, axis=1)
    return output_all

def regressionlinear(X,Y):
    regr = linear_model.LinearRegression()
    regr.fit(X, Y)
    #print('Intercept: \n', regr.intercept_)
    #print('Coefficients: \n', regr.coef_)

    X = sm.add_constant(X.to_numpy())
    model = sm.OLS(Y, X).fit()
    predictions = model.predict(X) 
 
    print_model = model.summary()
    #print(print_model)
    
    return regr.intercept_,regr.coef_



def randomforest(x_train,y_train,x_test,y_test):
    
    plt.rcParams['figure.dpi'] = 100

    regressor = RandomForestRegressor(n_estimators=200, max_depth=5)
    clf=regressor.fit(x_train, y_train)

    y_pred=regressor.predict(x_test)
    y_pred=pd.DataFrame(y_pred)


    plt_train=plt.scatter(x_train.iloc[:,0],y_train,   color='grey')
    plt_test=plt.scatter(x_test.iloc[:,0],y_test,   color='green')
    plt_pred=plt.scatter(x_test.iloc[:,0], y_pred,  color='black')

    plt.xlabel(x_test.columns[0])
    plt.ylabel(y_test.columns[0])
    plt.legend((plt_train, plt_test,plt_pred),("train data", "test data","prediction"))
    plt.show()

    print("Mean squared error: %.2f" % np.mean((regressor.predict(x_train) - y_train) ** 2))

    import seaborn as sns
    importances=regressor.feature_importances_

    indices=list(x_train)
    print("Feature ranking:")
    
    for f in range(x_train.shape[1]):
        print("Feature %s (%f)" % (indices[f], importances[f]))

    f, (ax1) = plt.subplots(1, 1, figsize=(6, 4), sharex=True)
    sns.barplot(indices, importances, palette="BrBG", ax=ax1)
    ax1.set_ylabel("Importance")

    ax1.set_xticklabels(
    ax1.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
    );

    return regressor.predict


@interact(model=['Bonny_light', 'Urals_NWE','Urals_MED','Tapis','Maya_USGC','Maya_Europe','WTI','Dalia','Dubai','Azeri_BTC'
                 ,'Arab_light','Arab_Heavy','Mars','ORL','Oseberg_blend'])

      
def NPV_Calc(model):
    
    
    
    if model == 'Bonny_light':
             
        plotdiag(combi,model)
        
                           
        #Non linear model
        nonlinear_BL = 0.0164924882990988*(DB) + 4.43302177278368e-5*np.power(DB,2) - 0.157317431833725
        #print("Non.linear predicted price =[", nonlinear_BL,"]")
        
        #Linear model
        Y = combi[['Bonny light']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent']].iloc[K[0]:]
        #print(X)
        #print(Y)
        intercept_, coef_ = regressionlinear(X,Y)
        linear_BL=intercept_+ coef_[0]*DB 
        #print("Linear predicted price =", linear_BL)
        
        #Random forest
               
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:,:]
        test =All.iloc[-100:,:]
        
        x_train=train["Dated Brent"].to_frame()
        y_train=train["Bonny light"]
        
        x_test=test["Dated Brent"].to_frame()
        y_test=test["Bonny light"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        

        #invar = {'Dated Brent':[DB]}
        invar_df = DB.to_frame()
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
       
        table = [linear_BL,nonlinear_BL,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)
              
      
        
    elif model == 'Urals_NWE':
        
        plotdiag(combi,model)
                
        #Non linear model
        nonlinear_UralN = 0.221579893076645*(FO35) + 0.0169570894526404*(DB) - 0.0154492977171762 
        - 0.0697233738254049*Go - 0.00856228790867718*Go*(FO35) - 0.00421761896551571*(FO35)**2
                
        #Linear model
        Y = combi[['Urals NWE']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','FO 3.5%',"Gasoil"]].iloc[K[0]:]
        X=X.fillna(0)               
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_UralN=intercept_+ coef_[0]*DB + coef_[1]*FO35 + coef_[2]*Go
                
        #Random forest
            
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[-100:,:]
        test =All.iloc[:-100,:]
        
        x_train=train[["Dated Brent","FO 3.5%","Gasoil"]]
        y_train=train["Urals NWE"]
        
        x_test=test[["Dated Brent","FO 3.5%","Gasoil"]]
        y_test=test["Urals NWE"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        

        invar_df = pd.concat([DB,FO35,Go],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_UralN,nonlinear_UralN,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)
        
    elif model == 'Tapis':    
        
        plotdiag(combi,model)
        
        #Non linear model
        nonlinear_Tapis = 0.569028548370371 + 0.158148187922098*Jet + 1.72847337855664e-6*np.power(DB,3)
        + 0.000198531291135097*Jet*np.power(ULSD,2) - 0.000213781975514373*np.power(ULSD,3)
        
        #Linear model
        Y = combi[['Tapis']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','Jet','ULSD']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_Tapis=intercept_+ coef_[0]*DB + coef_[1]*Jet + coef_[2]*ULSD
        
        #Random forest
               
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[-100:,:]
        test =All.iloc[:-100,:]
        
        x_train=train[["Dated Brent","Jet","ULSD"]]
        y_train=train["Tapis"]
        
        x_test=test[["Dated Brent","Jet","ULSD"]]
        y_test=test["Tapis"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
       
        invar_df = pd.concat([DB,Jet,ULSD],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_Tapis,nonlinear_Tapis,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)
               
        
    elif model == 'WTI':
        
        plotdiag(combi,model)
        
        #Non linear model
        nonlinear_WTI = -15.142388486653 - 2.06420352656404*Bu - 0.00877784990583735*DB*VGO05
        - 0.000867998048676867*np.power(Bu,3) - 0.0803378779475392*np.power(Bu,2)
        
        #Linear model
        Y = combi[['WTI']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','Butane','VGO 0.5%']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_WTI=intercept_+ coef_[0]*DB + coef_[1]*Bu + coef_[2]*VGO05
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[-80:,:]
        test =All.iloc[:80,:]
        
        x_train=train[["Dated Brent","Butane","VGO 0.5%"]]
        y_train=train["WTI"]
        
        x_test=test[["Dated Brent","Butane","VGO 0.5%"]]
        y_test=test["WTI"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        

        invar_df = pd.concat([DB,Bu,VGO05],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_WTI,nonlinear_WTI,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)       
        
    elif model == 'Maya_USGC':   
        
        plotdiag(combi,model)
        
        #nonlinear model
        nonlinear_MayaU =  0.687587246470687*FO35 + 0.00911247235417213*np.power(Jet,2) 
        + 0.0036123829112612*np.power(FO35,2) - 0.242991626831371*Jet

        #Linear model
        Y = combi[['Maya USGC']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','FO 3.5%','Jet']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_MayaU=intercept_+ coef_[0]*DB + coef_[1]*FO35 + coef_[2]*Jet
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:,:]
        test =All.iloc[:-100,:]
        
        x_train=train[["Dated Brent","FO 3.5%","Jet"]]
        y_train=train["Maya USGC"]
        
        x_test=test[["Dated Brent","FO 3.5%","Jet"]]
        y_test=test["Maya USGC"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        

        invar_df = pd.concat([DB,FO35,Jet],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        table = [linear_MayaU,nonlinear_MayaU,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)      
       
        
    elif model == 'Dubai':
        
        plotdiag(combi,model)
        
        #nonlinear model
        nonlinear_Dubai = 0.231614617140716*(FO35) + 0.0567664299088329*(DB) 
        + 0.000111685887466789*(DB)*np.power(FO35,2) - 1.13165773385408 
        - 3.92742103073686e-6*np.power(DB,3) - 0.0101251631604449*np.power(FO35,2)
        
        #Linear model
        Y = combi[['Dubai']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','FO 3.5%','FO 1%']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_Dubai=intercept_+ coef_[0]*DB + coef_[1]*FO35 + coef_[2]*FO1
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[-100:,:]
        test =All.iloc[:-100:]
        
        x_train=train[["Dated Brent","FO 3.5%","FO 1%"]]
        y_train=train["Dubai"]
        
        x_test=test[["Dated Brent","FO 3.5%","FO 1%"]]
        y_test=test["Dubai"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
      
        invar_df = pd.concat([DB,FO35,FO1],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
  
        table = [linear_Dubai,nonlinear_Dubai,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)    

    elif model == 'Azeri_BTC':
        
        plotdiag(combi,model)
        
        #Nonlinear model
        nonlinear_BTC = 2 - 0.0186675105114445*DB*np.exp(-7.82878945629328e-9*Po*Na*np.power(DB,3))
        #linear model
        Y = combi[['Azeri BTC']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','Propane','Naphtha']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_BTC=intercept_+ coef_[0]*DB + coef_[1]*Po + coef_[2]*Na
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:-50,:]
        test =All.iloc[-100:,:]
        
        x_train=train[["Dated Brent","Propane","Naphtha"]]
        y_train=train["Azeri BTC"]
        
        x_test=test[["Dated Brent","Propane","Naphtha"]]
        y_test=test["Azeri BTC"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
        invar_df = pd.concat([DB,Po,Na],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_BTC,nonlinear_BTC,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)
    
    elif model == 'Arab_light':
        
        plotdiag(combi,model)
        #Nonlinear model
        nonlinear_ArabL = 7.89067436674442 + 0.0158412920210973*Jet*(VGO2) 
        + 0.00760596666502651*np.power(DB,2)- 0.534520760449475*(DB)- 3.23806029482173e-5*np.power(DB,3) 
        - 1.39671145737361e-9*(VGO2)*np.power(DB,4)
        
        #Linearmodel
        Y = combi[['Arab light']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','Jet','VGO 2%','ULSD']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_ArabL=intercept_+ coef_[0]*DB + coef_[1]*Jet + coef_[2]*VGO2 + coef_[3]*ULSD
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:,:]
        test =All.iloc[:-100,:]
        
        x_train=train[["Dated Brent","Jet","VGO 2%","ULSD"]]
        y_train=train["Arab light"]
       
        x_test=test[["Dated Brent","Jet","VGO 2%","ULSD"]]
        y_test=test["Arab light"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
       
        invar_df = pd.concat([DB,Jet,VGO2,ULSD],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_ArabL,nonlinear_ArabL,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)    

    
        
    elif model == 'Arab_Heavy': 
        
        plotdiag(combi,model)
        #Nonlinear model
        nonlinear_ArabH = 0.635846243381231*(FO35) + 0.500212454466471*Jet + 13/Jet 
        + -0.0025957970628738/np.exp(0.635846243381231*(FO35) + 0.500212454466471*Jet) 
        - 5.90911403987227 - 0.0271336502751073*Jet*(FO35) - 0.0254885412647423*np.power(Jet,2)
        
        #Linear model
        Y = combi[['Arab Heavy']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent','FO 3.5%','Jet']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_ArabH=intercept_+ coef_[0]*DB + coef_[1]*FO35 + coef_[2]*Jet 
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:,:]
        test =All.iloc[-100:,:]
        
        x_train=train[["FO 3.5%","Jet"]]
        y_train=train["Arab Heavy"]
       
        x_test=test[["FO 3.5%","Jet"]]
        y_test=test["Arab Heavy"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
       
        invar_df = pd.concat([Fo35,Jet],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        table = [linear_ArabH,nonlinear_ArabH,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow) 
        
    elif model == 'Maya_Europe': 
        
        plotdiag(combi,model)
        #Nonlinear model
        nonlinear_MayaE =  0.494070816929047*(FO35) + 0.00213078770640727*Go**2 - 4.30356291903192

        #Linear model
        Y = combi[['Maya Europe']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['FO 3.5%','Gasoil',"FO 1%"]].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_MayaE=intercept_+ coef_[0]*FO35 + coef_[1]*Go + coef_[2]*FO1
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:,:]
        test =All.iloc[-100:,:]
        
        x_train=train[["FO 3.5%","Gasoil","FO 1%"]]
        y_train=train["Maya Europe"]
       
        x_test=test[["FO 3.5%","Gasoil","FO 1%"]]
        y_test=test["Maya Europe"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
        invar_df = pd.concat([FO35,Go,FO1],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
               
        table = [linear_MayaE,nonlinear_MayaE,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow) 
        
    elif model == 'Mars': 
        
        plotdiag(combi,model)
        
        #Nonlinear model
        nonlinear_Mars =  0.00533572984584189*Jet + 0.00232258205864726*Jet**2 - 5.0468498330526 
        - 0.379625720730507*(FO35) - 0.000535344925798245*(FO35)**3 - 0.0324550409045776*(FO35)**2

        #Linear model
        Y = combi[['Mars']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['FO 3.5%','Jet']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_Mars=intercept_+ coef_[0]*FO35 + coef_[1]*Jet 
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[-150:,:]
        test =All.iloc[:-100,:]
        
        x_train=train[["FO 3.5%","Jet"]]
        y_train=train["Mars"]
       
        x_test=test[["FO 3.5%","Jet"]]
        y_test=test["Mars"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
        invar_df = pd.concat([FO35,Jet],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_Mars,nonlinear_Mars,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)
        
    elif model == 'Urals_MED': 
        
        plotdiag(combi,model)
        
        #Nonlinear model
        nonlinear_UralM =  0.00724612659575245*(FO1)**2 + 0.00153428845373314*(FO1)**3 - 1.24167501962635 
        - 0.03181704251647*(FO35) - 0.210034973409967*(FO1) - 0.0295232993636577*(FO1)*(FO35) 
        - 0.00160888639770049*(FO35)*(FO1)**2
        
        #Linear model
        Y = combi[['Urals MED']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['FO 3.5%','FO 1%']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_UralM=intercept_+ coef_[0]*FO35 + coef_[1]*FO1
        
        
      
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[-150:,:]
        test =All.iloc[:-100,:]
        
        #x_train=train[["FO 3.5%","FO 1%"]]
        x_train=train[["FO 3.5%"]]
        y_train=train["Urals MED"]
       
        #x_test=test[["FO 3.5%","FO 1%"]]
        x_test=test[["FO 3.5%"]]
        y_test=test["Urals MED"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
             
        invar_df = pd.concat([FO35],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_UralM,nonlinear_UralM,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)   
          
        
    elif model == 'ORL': 
        
        plotdiag(combi,model)
        #Nonlinear model
        nonlinear_ORL = 3.26973453707456e-6*Na*Bu*Po**2 - 0.314833608268645 - 0.0162685919989057*Na*Bu
        
        #Linear model
        Y = combi[['ORL']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Naphtha','Butane','Propane']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_ORL=intercept_+ coef_[0]*Na + coef_[1]*Bu + coef_[2]*Po
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:,:]
        test =All.iloc[-100:,:]
        
        x_train=train[["Naphtha","Butane","Propane"]]
        y_train=train["ORL"]
       
        x_test=test[["Naphtha","Butane","Propane"]]
        y_test=test["ORL"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
              
        invar_df = pd.concat([Na,Bu,Po],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_ORL,nonlinear_ORL,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)

    elif model == 'Oseberg_blend': 
        
        plotdiag(combi,model)
        
        #Nonliner model
        nonlinear_Ose = 0.011666686468756*(DB) + 3.67495911112595e-7*np.power(DB,3)
        
        #linear model
        Y = combi[['Oseberg blend']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['Dated Brent']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_Ose=intercept_+ coef_[0]*DB
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:-70,:]
        test =All.iloc[-100:,:]
        
        x_train=train[["Dated Brent"]]
        y_train=train["Oseberg blend"]
       
        x_test=test[["Dated Brent"]]
        y_test=test["Oseberg blend"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
            
        invar_df = pd.concat([DB],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_Ose,nonlinear_Ose,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)
        
    elif model == 'Dalia':
        
        plotdiag(combi,model)
        #Nonliner model
        nonlinear_Dalia =  0.239431509820856*(FO1) + 0.00159358124012133*(DB)*(FO1)
        + 0.000155571867417853*(DB)*(FO1)**2
        
        #linear model
        Y = combi[['Dalia']].dropna()
        K=list(Y.index.values)
        K[0]
        X = combi[['FO 1%','Dated Brent']].iloc[K[0]:]
        X = X.fillna(0)       
        intercept_, coef_ = regressionlinear(X,Y)
        coef_ = coef_.reshape(-1)
        linear_Dalia=intercept_+ coef_[0]*FO1 + coef_[1]*DB 
        
        #Random forest
        All = pd.concat([X,Y],axis=1,sort=False)
        train = All.iloc[:,:]
        test =All.iloc[-100:,:]
        
        x_train=train[["FO 1%","Dated Brent"]]
        y_train=train["Dalia"]
       
        x_test=test[["FO 1%","Dated Brent"]]
        y_test=test["Dalia"].to_frame()
        F=randomforest(x_train,y_train,x_test,y_test)
        
               
        invar_df = pd.concat([DB,FO1],axis = 1)
        y_pred = F(invar_df)
        y_pred=pd.Series(y_pred)
        
        
        table = [linear_Dalia,nonlinear_Dalia,y_pred]
        tableshow = pd.concat(table, axis =1)
        tableshow.columns = ['Linear','Nonlinear','Randomforest']
        display(tableshow)
        
        
    else:
        print('choose something from drop down menu')   

    return
    








interactive(children=(Dropdown(description='model', options=('Bonny_light', 'Urals_NWE', 'Urals_MED', 'Tapis',…